In [1]:
from aocd import get_data, submit
from operator import itemgetter

In [2]:
example_data = ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"
real_data = get_data(day=17, year=2022)

In [3]:
state_mapping = [1 << i for i in range(7)]


def is_blocked(row, column, states, cropped):
    if len(states) - 1 < row - cropped:
        return False
    return states[row - cropped] & state_mapping[column]


def crop_states(states):
    last = len(states) - 1
    for i in range(1, len(states)):
        if states[last - i] | states[last - i + 1] == (1 << 7) - 1:
            return last - i, states[last - i:]

    return 0, states


class Rock:
    def __init__(self, position, elements):
        self.position = position
        self.elements = elements
        self.max_right = max(elements, key=itemgetter(1))[1]
        self.max_left = min(elements, key=itemgetter(1))[1]

    def can_push_right(self, states, cropped):
        if self.position[1] + self.max_right >= 6:
            return False
        for element in self.elements:
            column = self.position[1] + element[1]
            row = self.position[0] + element[0]

            if is_blocked(row, column + 1, states, cropped):
                return False

        return True

    def can_push_left(self, states, cropped):
        if self.position[1] + self.max_left <= 0:
            return False

        for element in self.elements:
            column = self.position[1] + element[1]
            row = self.position[0] + element[0]
            if is_blocked(row, column - 1, states, cropped):
                return False
        return True

    def can_push_down(self, states, cropped):
        for element in self.elements:
            column = self.position[1] + element[1]
            row = self.position[0] + element[0]

            if row <= 0 or is_blocked(row - 1, column, states, cropped):
                return False

        return True

    def push_right(self):
        self.position = [self.position[0], self.position[1] + 1]

    def push_left(self):
        self.position = [self.position[0], self.position[1] - 1]

    def push_down(self):
        self.position = [self.position[0] - 1, self.position[1]]

    def update_states(self, states, cropped):
        for element in self.elements:
            column = self.position[1] + element[1]
            row = self.position[0] + element[0]

            while len(states) - 1 < row - cropped:
                states.append(0)

            states[row - cropped] = states[row - cropped] | state_mapping[column]

        return states


In [4]:
def create_rock(row, iteration):
    if iteration % 5 == 0:
        return Rock([row + 4, 2], [[0, 0], [0, 1], [0, 2], [0, 3]])

    elif iteration % 5 == 1:
        return Rock([row + 4, 3], [[0, 0], [1, -1], [1, 0], [1, 1], [2, 0]])

    elif iteration % 5 == 2:
        return Rock([row + 4, 2], [[0, 0], [0, 1], [0, 2], [1, 2], [2, 2]])

    elif iteration % 5 == 3:
        return Rock([row + 4, 2], [[0, 0], [1, 0], [2, 0], [3, 0]])

    elif iteration % 5 == 4:
        return Rock([row + 4, 2], [[0, 0], [0, 1], [1, 0], [1, 1]])

In [5]:
def visualize(states, rock, iteration):
    print("---------------", iteration)
    for state in reversed(states):
        text_state = f"{state:07b}"[::-1]
        text_state = text_state.replace("0", ".")
        print(text_state)


def simulate(data, iterations=2022):
    privious_states = dict()
    states = []
    movement_index = 0
    cropped = 0

    detect_cycle = True

    iteration = 0
    while iteration < iterations:

        if iteration % 5 == 0 and detect_cycle:
            for i in privious_states:
                if privious_states[i][1] == states and privious_states[i][2] == movement_index % len(data):
                    cycle_length = iteration - i
                    added_rows = cropped - privious_states[i][0]
                    add_cycles = ((iterations - iteration) // cycle_length)

                    if True:
                        iteration = iteration + add_cycles * cycle_length
                        cropped = cropped + add_cycles * added_rows
                        detect_cycle = False
                        break

        if detect_cycle:
            privious_states[iteration] = (cropped, states.copy(), movement_index % len(data))

        rock = create_rock(len(states) - 1 + cropped, iteration)

        while True:
            movement = data[movement_index % len(data)]
            movement_index = movement_index + 1

            if movement == "<" and rock.can_push_left(states, cropped):
                rock.push_left()
            elif movement == ">" and rock.can_push_right(states, cropped):
                rock.push_right()

            if rock.can_push_down(states, cropped):
                rock.push_down()
            else:
                states = rock.update_states(states, cropped)
                add_cropped, states = crop_states(states)

                cropped = cropped + add_cropped

                # visualize(states, rock, iteration)
                break

        iteration = iteration + 1

    return len(states) + cropped


print(f"Height after 2022 rounds: {simulate(real_data, 2022)}")

Height after 2022 rounds: 3191


In [6]:
print(f"Height after 1000000000000 rounds: {simulate(real_data, 1000000000000)}")

Height after 1000000000000 rounds: 1572093023267
